# 이미지 데이터 전처리

In [5]:
from tensorflow import keras
from tensorflow.keras import layers
from tensorflow.keras.preprocessing.image import ImageDataGenerator

In [8]:
BATCH_SIZE = 32
image_height = 224
image_width = 224
train_dir = '../catdog/train'
test_dir = '../catdog/test'

train = ImageDataGenerator(
    rescale = 1./255,
    rotation_range = 10,
    width_shift_range=0.1,
    height_shift_range=0.1,
    shear_range=0.1,
    zoom_range=0.1
    )
train_generator = train.flow_from_directory(train_dir,
                                            target_size = (image_height, image_width),
                                            color_mode = 'rgb',
                                            batch_size=BATCH_SIZE,
                                            seed=1,
                                            shuffle=True,
                                            class_mode="binary")

Found 3680 images belonging to 2 classes.


In [13]:
img, label = next(train_generator)

In [14]:
img

array([[[[0.04519807, 0.05146727, 0.0206949 ],
         [0.04542368, 0.05109125, 0.02167255],
         [0.04564929, 0.05071523, 0.02265021],
         ...,
         [0.5172181 , 0.61242104, 0.33399436],
         [0.28986505, 0.30705705, 0.2506161 ],
         [0.2441497 , 0.2483241 , 0.15408063]],

        [[0.04313726, 0.0690706 , 0.03655982],
         [0.04313726, 0.06876979, 0.03603339],
         [0.04313726, 0.06846897, 0.03550697],
         ...,
         [0.5122683 , 0.6055963 , 0.33339438],
         [0.2881401 , 0.3051821 , 0.24694122],
         [0.24497469, 0.24877408, 0.15588057]],

        [[0.2818179 , 0.29771984, 0.2932951 ],
         [0.27715528, 0.2932828 , 0.28833166],
         [0.27249265, 0.28884578, 0.28336817],
         ...,
         [0.31046218, 0.34054813, 0.20527382],
         [0.22349563, 0.26792124, 0.1840693 ],
         [0.32626435, 0.35422757, 0.24869274]],

        ...,

        [[0.5921569 , 0.60784316, 0.654902  ],
         [0.5921569 , 0.60784316, 0.654902  ]

In [15]:
label

array([1., 1., 0., 1., 1., 1., 1., 1., 1., 1., 0., 1., 0., 1., 1., 1., 0.,
       1., 1., 1., 1., 0., 1., 0., 1., 0., 0., 1., 0., 0., 0., 0.],
      dtype=float32)